# Introduction

## Using Jupyter

Jupyter is a front end to the [IPython](https://ipython.org) interactive shell, and offers IDE like features.  It is separated into two main types of cell: [Markdown](https://en.wikipedia.org/wiki/Markdown) cells (such as this one) which allows markdown or HTML code to be written, and code cells like the next one which can be run in real time.

To execute code in a cell, press `Crtl` + `Enter`, click on the `[ > ]Run` button in the main menu, or press `Shift` + `Enter` if you wish to execute the code and then move on to a new cell (creating it if it does not already exist).

## Setup

MongoDB is a NoSQL database, which has a core API in JavaScript, and a series of other APIs in different languages.  The one we are going to use is the Python API, [PyMongo](https://api.mongodb.com/python/current/).  MongoDB instance on your VM is already started by default.

PyMongo is a package that contains tools to work with MongoDB from Python. We have installed it in the VM provided to you. 

This imports the `MongoClient` class from the pymongo module, which we will use to deal with all our connections from.  We're connecting to our localhost, which is listening on port 27017. There are more options, the documentation for the formatting of the connection string is at https://docs.mongodb.com/manual/reference/connection-string/.

In [2]:
!pip install pymongo

    100% |████████████████████████████████| 409kB 1.5MB/s ta 0:00:01
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')

If you do not get any errors, you have confirmed PyMongo library has been successfully installed&configured in the VM. We will now check to see whether the connection is correct.  The following code calls a function which returns a list of all current databases.  If your Mongo instance is still empty, it should be something like `['admin', 'local']`.

In [3]:
client.list_database_names()

[u'local']

Next, we are going to create a database object `db`, which is a property of the `client` object.  MongoDB is schemaless, and so accessing a database like this will create the database if it does not already exist.

A database can be accessed by using "dot" notation (i.e., `client.dbname`), or dictionary notation (i.e., `client['dbname']`).  This also applies to making collections

Create a database called `test` in a variable called `db`.  Using that variable, create a collection called `test_collection` with a variable called `collection` as follows.  Run the code in the following cell (there should not be any output)

In [4]:
# Create database and collection objects for convenience
db = client.mongo_lab
collection = db.test_collection

# Using MongoDB

## Inserting data

MongoDB data is stored as BSON (binary JSON), which is essentially JSON with some additional optimisations, so the way to insert data is as a JSON object.  For Python, you can use a `dict` or a `list` for this, and then call either `insert_one` or `insert_many` on the collection.  

In [5]:
# Create an object and insert into the `test_collection`
single_obj = {'name': 'Amber', 'star_sign': 'Capricorn', 'favourite_song': 'The Load-Out'}
collection.insert_one(single_obj)
single_obj_2 = {'name': 'Huw', 'star_sign': 'Libra', 'favourite_song': 'The masses against the classes'}
collection.insert_one(single_obj_2)
single_obj_3 = {'name': 'Robert', 'star_sign': 'Leo', 'favourite_song': 'Bad day'}
collection.insert_one(single_obj_3)

We will look at querying data in more detail below, but for now, to see whether the object got successfully inserted into the collection, run the code below.  This will always return the first instance which matches the query.  You will notice that even though we didn't specify `_id` one got added already.  This is a unique identifier for the document in the collection

In [6]:
collection.find_one() # returns a single document matching the query condition

{u'_id': ObjectId('5c0bd1905ccae90221cefffb'),
 u'favourite_song': u'The Load-Out',
 u'name': u'Amber',
 u'star_sign': u'Capricorn'}

In [7]:
collection.find_one({'name':'Robert'})

{u'_id': ObjectId('5c0bd1905ccae90221cefffd'),
 u'favourite_song': u'Bad day',
 u'name': u'Robert',
 u'star_sign': u'Leo'}

Remember, for MongoDB, you do not have to specify a schema or create a collection, it will be created automatically.  You don't need to keep to the same layout, but can have entirely different objects.  Consider the following: 

In [8]:
from datetime import datetime
obj1 = {'Meaning of life': 42}
obj2 = {'ABC': 'DEF', 'time': datetime.now()}
collection.insert_one(obj1)
collection.insert_one(obj2)

We can also use the `insert_many`, which accepts a list of dicts.  In the cell below, create a list of dicts called `many_objects`, and call the `insert_many` function.  The code below that will iterate over all the documents in the database.

In [9]:
from pprint import pprint

In [10]:
# YOUR CODE HERE

list_of_dicts = [
    {
        "name": "Mini",
        "age": 23,
        "planet": "unknown"
    },
    {
        "name": "Paddy",
        "age": 25
    },
    {
        "name": "Saurav",
        "planet": "Earth"
    }
]

# collection.insert_many(list_of_dicts)


#See what has been inserted into the collection
for doc in collection.find():
    pprint(doc)

{u'_id': ObjectId('5c0bd1905ccae90221cefffb'),
 u'favourite_song': u'The Load-Out',
 u'name': u'Amber',
 u'star_sign': u'Capricorn'}
{u'_id': ObjectId('5c0bd1905ccae90221cefffc'),
 u'favourite_song': u'The masses against the classes',
 u'name': u'Huw',
 u'star_sign': u'Libra'}
{u'_id': ObjectId('5c0bd1905ccae90221cefffd'),
 u'favourite_song': u'Bad day',
 u'name': u'Robert',
 u'star_sign': u'Leo'}
{u'Meaning of life': 42, u'_id': ObjectId('5c0bd1905ccae90221cefffe')}
{u'ABC': u'DEF',
 u'_id': ObjectId('5c0bd1905ccae90221ceffff'),
 u'time': datetime.datetime(2018, 12, 8, 14, 13, 36, 504000)}


## Importing and querying data

For this part of the exercise, we will use a sample dataset provided by Mongo  for a documentation tutorial.  The following cell runs the `mongoimport` command, which is a Unix command which comes with Mongo for importing data. We will need to run a bash command in the next cell first.  This uses the Jupyter \"magics\", and requires that the first line include `%%bash`. The code does the following:

- Download the JSON file from the url, and save as ./primer-dataset.json
- Import into the `test` database into the collection `restaurants` whilst dropping any collection which already exists from the file ./primer-dataset.json
- Deletes the file.

 Click on the following cell, and execute it:

In [11]:
%%bash
# Use wget to download the data
wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
# mongoimport is the Mongo command to import data.  
# It specifies the database, collection and format, and import file
# --drop means it's going to drop any collection with the same name which already exists
mongoimport --db mongo_lab --collection restaurants --drop --file ./primer-dataset.json
# Delete the JSON file we just downloaded
rm ./primer-dataset.json

--2018-12-08 14:13:36--  https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11874761 (11M) [text/plain]
Saving to: ‘primer-dataset.json’

     0K .......... .......... .......... .......... ..........  0% 9.77M 1s
    50K .......... .......... .......... .......... ..........  0% 22.5M 1s
   100K .......... .......... .......... .......... ..........  1% 19.9M 1s
   150K .......... .......... .......... .......... ..........  1% 20.7M 1s
   200K .......... .......... .......... .......... ..........  2% 17.8M 1s
   250K .......... .......... .......... .......... ..........  2% 15.0M 1s
   300K .......... .......... .......... .......... ..........  3% 16.2M 1s
   350K .......... .......... .......... .......... .

Change the variable `collection` to refer to the new collection `restaurants`, and inspect the general format of the data by adding the code below to find the first record of the collection:

In [12]:
# YOUR CODE HERE

collection = db.restaurants

We saw the [`collection.find()`](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find) function earlier to return all the documents we inserted into our `test` collection.  Without any arguments, `find()` will return a cursor of all the available documents from in the collection.  To refine queries, however, the search can be filtered by the addition of a first parameter.

The `filter` parameter is a dict, which searches for the documents where `key` = `value` where the dict is of the form `{key: value}`.  For example, to find all bakeries in the city, we would do the following query:   

**WARNING** Unlike the Mongo command line interface, if you try and print the output of a `find()` query, it will continue to output all results until it has finished.  This can cause the browser to crash, particularly if it is a particularly large query set.

Using `find().count()` is a useful way of checking how many a result will return, and `find_one()` to see the general structure of a result. If you use `find()`, make sure you either include the `limit` argument, or have a counter or other condition to break out of your printing loop!

In [13]:
for doc in collection.find({'cuisine': 'Bakery'}).limit(5):
    pprint(doc)

{u'_id': ObjectId('5c0bd192d7994444426c6afe'),
 u'address': {u'building': u'1007',
              u'coord': [-73.856077, 40.848447],
              u'street': u'Morris Park Ave',
              u'zipcode': u'10462'},
 u'borough': u'Bronx',
 u'cuisine': u'Bakery',
 u'grades': [{u'date': datetime.datetime(2014, 3, 3, 0, 0),
              u'grade': u'A',
              u'score': 2},
             {u'date': datetime.datetime(2013, 9, 11, 0, 0),
              u'grade': u'A',
              u'score': 6},
             {u'date': datetime.datetime(2013, 1, 24, 0, 0),
              u'grade': u'A',
              u'score': 10},
             {u'date': datetime.datetime(2011, 11, 23, 0, 0),
              u'grade': u'A',
              u'score': 9},
             {u'date': datetime.datetime(2011, 3, 10, 0, 0),
              u'grade': u'B',
              u'score': 14}],
 u'name': u'Morris Park Bake Shop',
 u'restaurant_id': u'30075445'}
{u'_id': ObjectId('5c0bd192d7994444426c6b20'),
 u'address': {u'building':

Noted, count() is deprecated in the MongoDb drivers compatible with the 4.0features, as a result, we should use countDocuments() in the following exercises. (for more information, check https://docs.mongodb.com/manual/reference/method/db.collection.count/ and http://api.mongodb.com/python/current/changelog.html)

In [14]:
collection.count_documents({'cuisine': 'Bakery'})

691

A filter can have as many conditions as you like, and will assume that you are using an AND condition, unless you specify otherwise (as below).  In the cell below, write a query to return the number/count of all the establishments with a cuisine of `Hamburgers` in the borough of Manhattan.

In [15]:
# YOUR CODE HERE
# All establishments with: 
# * a cuisine of 'Hamburgers' 
# * in the borough of 'Manhattan

collection.count_documents({'cuisine': 'Hamburgers', 'borough':'Manhattan'})



124

In [16]:
collection.distinct('cuisine')

[u'Bakery',
 u'Hamburgers',
 u'Irish',
 u'American',
 u'Jewish/Kosher',
 u'Delicatessen',
 u'Ice Cream, Gelato, Yogurt, Ices',
 u'Chinese',
 u'Other',
 u'Chicken',
 u'Turkish',
 u'Caribbean',
 u'Donuts',
 u'Sandwiches/Salads/Mixed Buffet',
 u'Bagels/Pretzels',
 u'Continental',
 u'Pizza',
 u'Italian',
 u'Steak',
 u'Polish',
 u'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 u'German',
 u'French',
 u'Pizza/Italian',
 u'Mexican',
 u'Spanish',
 u'Caf\xe9/Coffee/Tea',
 u'Tex-Mex',
 u'Pancakes/Waffles',
 u'Soul Food',
 u'Seafood',
 u'Hotdogs',
 u'Greek',
 u'Not Listed/Not Applicable',
 u'African',
 u'Japanese',
 u'Indian',
 u'Armenian',
 u'Thai',
 u'Chinese/Cuban',
 u'Mediterranean',
 u'Korean',
 u'Bottled beverages, including water, sodas, juices, etc.',
 u'Russian',
 u'Eastern European',
 u'Middle Eastern',
 u'Asian',
 u'Ethiopian',
 u'Vegetarian',
 u'Barbecue',
 u'Egyptian',
 u'English',
 u'Sandwiches',
 u'Portuguese',
 u'Indonesian',
 u'Chinese/Japanese',
 u'Filipino'

### Sub-documents

A valid JSON style "document" can have another JSON document inside it.  To access these, we use the "dot" notation to access them.  For example, to get all the restaurants in a certain zipcode, you would run code as follows:

In [17]:
from pprint import pprint
cursor = collection.find({'address.zipcode': '10462'}, {'address.zipcode': 1, '_id':0}).limit(5)
for c in cursor:
    pprint(c)

{u'address': {u'zipcode': u'10462'}}
{u'address': {u'zipcode': u'10462'}}
{u'address': {u'zipcode': u'10462'}}
{u'address': {u'zipcode': u'10462'}}
{u'address': {u'zipcode': u'10462'}}


### Operators

MongoDB has a series of [operators](https://docs.mongodb.com/manual/reference/operator/query/) which allow us to do more sophisticated filters on our queries.  There are too many to go into individually, but we will look at a few important ones.  The specific syntax varies depending on the operator, so it isn't possible to give a general rule, but we will go over a few examples here.  Make sure you check the [documentation](https://docs.mongodb.com/manual/reference/operator/query/) for use on each one.

#### [\$or](https://docs.mongodb.com/manual/reference/operator/query/or/#op._S_or)

Performs a logical **OR** operation on all the key/value pairs in a list, as in the code below:

In [18]:
filter = {"$or": [{"cuisine": "Polynesian"}, {"cuisine": "Hawaiian"}]}
for f in collection.find(filter, {'name': 1, '_id': 0}).limit(2):
    pprint(f)

{u'name': u'Camaradas El Barrio'}
{u'name': u'Makana'}


#### [`$regex`](https://docs.mongodb.com/manual/reference/operator/query/regex/#op._S_regex)

The `$regex` operator searches for a regular expression on a particular field.  Within the filter field, the named field (a key) takes a dict as a value.  

For example, to search for all restaurants which start with the word "Pretzel" in the title you can do the following:

In [19]:
filter = {"name": {"$regex": '^Pretzel'}}
collection.count_documents(filter)

4

There are other ways to use regular expressions in PyMongo, you can use the [`re`](https://docs.python.org/3/library/re.html) module in Python.  In Mongo itself, you can use the following syntax: The simplest is to enclose the regular expression inside `/` characters, as in `{"name": /^Pretzel/}`, but that doesn't work properly in PyMongo.

Using the `$regex` operator, find all restaurants which end in "Bar" in the borough of Brooklyn.

HINT: The regex character for the end of a string is `$`

In [20]:
# YOUR CODE HERE
filter_regex = {"name": {"$regex": "Bar$"}}

collection.count_documents(filter_regex)

653

#### [`$gt`](https://docs.mongodb.com/manual/reference/operator/query/gt/#op._S_gt)

The `$gt` operator is a comparison between two values where one is greater than the other.  

For example, consider this code which finds restaurants which have had a score of more than 15:

In [21]:
filter = {'grades.score': {'$gt': 12}}
collection.count_documents(filter)

13980

Using one of the other comparison operators, find all restaurants which had a grade awarded on the 15 December 2012.  You'll need to create a [`datetime`](https://docs.python.org/2/library/datetime.html#datetime-objects) object in Python.

In [22]:
# YOUR CODE HERE

import datetime

date_to_filter = datetime.datetime(2012, 12, 15)

for doc in collection.find({'grades.date': date_to_filter}, {'_id': 0, 'grades': 1, 'name': 1}):
    pprint (doc)

{u'grades': [{u'date': datetime.datetime(2014, 8, 5, 0, 0),
              u'grade': u'A',
              u'score': 10},
             {u'date': datetime.datetime(2014, 2, 5, 0, 0),
              u'grade': u'A',
              u'score': 12},
             {u'date': datetime.datetime(2012, 12, 15, 0, 0),
              u'grade': u'A',
              u'score': 12},
             {u'date': datetime.datetime(2011, 11, 23, 0, 0),
              u'grade': u'A',
              u'score': 12}],
 u'name': u'Ajanta India'}
{u'grades': [{u'date': datetime.datetime(2014, 12, 16, 0, 0),
              u'grade': u'A',
              u'score': 4},
             {u'date': datetime.datetime(2013, 12, 23, 0, 0),
              u'grade': u'A',
              u'score': 9},
             {u'date': datetime.datetime(2012, 12, 15, 0, 0),
              u'grade': u'A',
              u'score': 10}],
 u'name': u'Internationalhouse'}
{u'grades': [{u'date': datetime.datetime(2015, 1, 14, 0, 0),
              u'grade': u'A',
      

## Organising output

So far, we have seen the two of the arguments in the `find()` and related functions.  The `filter` which allows us to select the criteria for documents in the collection, and the `limit` to limit the amount of results.  You should read the documentation fully about the function in your own time, but for now, we will go over two other arguments which are for organising output: field selection, and sorting.

The field selection or `projection` argument is the argument after the \[optional\] filter, and is either:

* A list of fields to include (plus \_id)
* A dict of fields with True/False to include

For example, to display only the name of the restaurant:

In [23]:
filter = {'cuisine': 'Brazilian'}
fields = {'_id': False, 'name': True}
collection.find_one(filter, fields)

{u'name': u'Churrascaria Plataforma'}

The sort argument is a dict object of field names as keys, and directions.  This can be done either as a named parameter when calling `find()`, or as a function in its own right [`sort()`](https://api.mongodb.com/python/current/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort)

For example, to sort in alphabetical order, consider the following code:

In [24]:
import pymongo
# The ASCENDING and DESCENDING constants have values of 1 (ASCENDING) and -1 (DESCENDING)
sort = [('name', pymongo.ASCENDING)]
for d in collection.find(filter, projection=fields, sort=sort):
    pprint(d)

{u'name': u'Barzinho'}
{u'name': u'Beco'}
{u'name': u'Beija-Flor'}
{u'name': u'Berimbau'}
{u'name': u'Brazil Brazil Restuarant'}
{u'name': u'Brazil Grill'}
{u'name': u'Buffet 58'}
{u'name': u'Carioca Grill'}
{u'name': u'Casa'}
{u'name': u'Churrascaria Plataforma'}
{u'name': u'Circus Restaurant'}
{u'name': u'Copacabana Pizza & Grill'}
{u'name': u'Emporium Brasil Restaurant'}
{u'name': u'Esperanto'}
{u'name': u'Fogo De Chao'}
{u'name': u'Malagueta  Restaurant'}
{u'name': u'Miss Favela'}
{u'name': u'Pacificos Fine Foods'}
{u'name': u'Point Brazil'}
{u'name': u'Rainhas'}
{u'name': u'Rice & Beans Restaurant'}
{u'name': u'Sao Restaurant'}
{u'name': u'Texas De Brazil Churrascaria'}
{u'name': u'Via Brazil'}
{u'name': u'Villa Brazil Cafe Grill'}
{u'name': u'Zebu Grill'}


# MongoDB Aggregation Framework

The most common usage for the aggregation framework is to perform group operations such as sum, count or average.  The framework works as a pipeline, with a series of different stages where the data are transformed in each one.

At its simplest, this can be used to obtain output like min, max, count, avg on a collection as follows:

[link](https://docs.mongodb.com/manual/reference/operator/aggregation/group/)

In [25]:
group = {
    '$group': {
        '_id': None, 
        'size': {'$sum': 1},
        'min': {'$min': '$restaurant_id'},
        'max': {'$max': '$restaurant_id'}
    }
}

cursor = collection.aggregate([group])
for c in cursor:
    print(c)

{u'size': 25359, u'max': u'50018995', u'_id': None, u'min': u'30075445'}


Note that it has an `$_id: None` key/value pair in it.  It is compulsory for a `$group` pipeline to have one, and it indicates what it is grouping by.  In this case, we haven't grouped it at all, however it can also be used for more complex output where documents are grouped according to a field.

### Aggregation example

Consider this example, of finding the breakdown of how many of each type of restaurant there is in the Bronx.  We would need to go through the following stages:

- Identify restaurants which are in the Bronx
- Group the restaurants by type to get the count
- Sort the results in a sensible way

The code to perform this query is below:

In [26]:
# Restrict the results to only establishments in the Bronx.  
# '$match' indicates the stage in the pipeline, and the dictionary is the same as using with find()
match = {
    "$match": {"borough": "Bronx"}
}

# $group indicates the stage in the pipeline
# _id is the field to perform the operation on (like SQL GROUP BY)
# count is the name of the field that the result will be in
# $sum is the counting operation, and the value 1 is how many to count each time
group = {
    '$group': {'_id': '$cuisine', 'count': {'$sum': 1}}
    
}
# $sort indicates the position in the pipeline
# count is the field to sort by, and -1 means to sort in descending order
sort = {
    '$sort': {'count': pymongo.DESCENDING}
}

match_final = {
    "$match": {"count": {"$gt": 100}}
}

cursor = collection.aggregate([match, group, sort, match_final])
for c in cursor:
    print(c)



{u'count': 411, u'_id': u'American'}
{u'count': 323, u'_id': u'Chinese'}
{u'count': 197, u'_id': u'Pizza'}
{u'count': 187, u'_id': u'Latin (Cuban, Dominican, Puerto Rican, South & Central American)'}
{u'count': 127, u'_id': u'Spanish'}
{u'count': 110, u'_id': u'Caribbean'}
{u'count': 108, u'_id': u'Chicken'}


This is a simple query, which shows some of the basic stages of the aggregation pipeline.  It can be improved as follows:

* We can change the name of the `_id` in the output back to `cuisine` using the `$project` stage
* We can change the order of the output to be sorted in alphabetical order as well
* We can limit the results to include results only with a count of 20 or more

Implement those stages in the cell below

In [27]:
# YOUR CODE HERE

project_stage = {
    "$project": {"cuisine": "$_id", "count": 1}
}

sort_stage = {
    "$sort": {"cuisine": pymongo.ASCENDING}
}

match_stage = {
    "$match": {"count": {"$gt":20} }
}

pipeline = [match, group, project_stage, match_stage, sort_stage] #More code here...
cursor = collection.aggregate(pipeline)
for c in cursor:
    print(c)


{u'count': 31, u'cuisine': u'African', u'_id': u'African'}
{u'count': 411, u'cuisine': u'American', u'_id': u'American'}
{u'count': 71, u'cuisine': u'Bakery', u'_id': u'Bakery'}
{u'count': 45, u'cuisine': u'Caf\xe9/Coffee/Tea', u'_id': u'Caf\xe9/Coffee/Tea'}
{u'count': 110, u'cuisine': u'Caribbean', u'_id': u'Caribbean'}
{u'count': 108, u'cuisine': u'Chicken', u'_id': u'Chicken'}
{u'count': 323, u'cuisine': u'Chinese', u'_id': u'Chinese'}
{u'count': 26, u'cuisine': u'Delicatessen', u'_id': u'Delicatessen'}
{u'count': 68, u'cuisine': u'Donuts', u'_id': u'Donuts'}
{u'count': 78, u'cuisine': u'Hamburgers', u'_id': u'Hamburgers'}
{u'count': 27, u'cuisine': u'Ice Cream, Gelato, Yogurt, Ices', u'_id': u'Ice Cream, Gelato, Yogurt, Ices'}
{u'count': 52, u'cuisine': u'Italian', u'_id': u'Italian'}
{u'count': 35, u'cuisine': u'Juice, Smoothies, Fruit Salads', u'_id': u'Juice, Smoothies, Fruit Salads'}
{u'count': 187, u'cuisine': u'Latin (Cuban, Dominican, Puerto Rican, South & Central American)'

### Detour: Aggregation with the Zip Code Data Set

[link](https://docs.mongodb.com/manual/tutorial/aggregation-zip-code-data-set/)


#### Import data

In [28]:
%%bash
# Use wget to download the data
wget http://media.mongodb.org/zips.json
# mongoimport is the Mongo command to import data.  
# It specifies the database, collection and format, and import file
# --drop means it's going to drop any collection with the same name which already exists
mongoimport --db mongo_lab --collection zipcodes --drop --file ./zips.json
# Delete the JSON file we just downloaded
rm ./zips.json

--2018-12-08 14:13:39--  http://media.mongodb.org/zips.json
Resolving media.mongodb.org (media.mongodb.org)... 13.32.123.22, 13.32.123.202, 13.32.123.33, ...
Connecting to media.mongodb.org (media.mongodb.org)|13.32.123.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3182409 (3.0M) [application/json]
Saving to: ‘zips.json’

     0K .......... .......... .......... .......... ..........  1% 7.94M 0s
    50K .......... .......... .......... .......... ..........  3% 8.84M 0s
   100K .......... .......... .......... .......... ..........  4% 24.8M 0s
   150K .......... .......... .......... .......... ..........  6% 6.41M 0s
   200K .......... .......... .......... .......... ..........  8% 6.39M 0s
   250K .......... .......... .......... .......... ..........  9% 1.16M 1s
   300K .......... .......... .......... .......... .......... 11% 6.58M 1s
   350K .......... .......... .......... .......... .......... 12% 25.6M 1s
   400K .......... .......... ........

In [29]:
db.collection_names()

/home/comp6235/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


[u'restaurants', u'zipcodes', u'test_collection', u'system.indexes']

Examining structure of a document

In [30]:
collection = db.zipcodes

collection.find_one({})

{u'_id': u'01001',
 u'city': u'AGAWAM',
 u'loc': [-72.622739, 42.070206],
 u'pop': 15338,
 u'state': u'MA'}

### Return States with Populations above 10 Million

In [31]:
group_stage = {
    "$group": {"_id": "$state", "totalPop": { "$sum": "$pop" }}
}

match_stage = {
    "$match": {"totalPop": {"$gte": 10*1000*1000 }}
}

pipeline = [group_stage, match_stage]

for doc in collection.aggregate(pipeline):
    pprint(doc)

{u'_id': u'CA', u'totalPop': 29754890}
{u'_id': u'FL', u'totalPop': 12686644}
{u'_id': u'TX', u'totalPop': 16984601}
{u'_id': u'NY', u'totalPop': 17990402}
{u'_id': u'PA', u'totalPop': 11881643}
{u'_id': u'OH', u'totalPop': 10846517}
{u'_id': u'IL', u'totalPop': 11427576}


### Return Average City Population by State

Output:

```

{
  "_id" : "MN",
  "avgCityPop" : 5335
}

```

The first \$group stage groups the documents by the combination of city and state, uses the $sum expression to calculate the population for each combination, and outputs a document for each city and state combination.

In [32]:
group_stage1 = {
    "$group": { "_id": {"state": "$state", "city": "$city"}, "pop": { "$sum": "$pop" }}
}


pipeline = [group_stage1]

for doc in collection.aggregate(pipeline):
    pprint(doc)

{u'_id': {u'city': u'WRANGELL', u'state': u'AK'}, u'pop': 2573}
{u'_id': {u'city': u'KLAWOCK', u'state': u'AK'}, u'pop': 851}
{u'_id': {u'city': u'HYDER', u'state': u'AK'}, u'pop': 116}
{u'_id': {u'city': u'HYDABURG', u'state': u'AK'}, u'pop': 891}
{u'_id': {u'city': u'KETCHIKAN', u'state': u'AK'}, u'pop': 14308}
{u'_id': {u'city': u'SKAGWAY', u'state': u'AK'}, u'pop': 692}
{u'_id': {u'city': u'HOONAH', u'state': u'AK'}, u'pop': 1670}
{u'_id': {u'city': u'HAINES', u'state': u'AK'}, u'pop': 2246}
{u'_id': {u'city': u'DOUGLAS', u'state': u'AK'}, u'pop': 1802}
{u'_id': {u'city': u'ANGOON', u'state': u'AK'}, u'pop': 1002}
{u'_id': {u'city': u'NUIQSUT', u'state': u'AK'}, u'pop': 354}
{u'_id': {u'city': u'WAINWRIGHT', u'state': u'AK'}, u'pop': 492}
{u'_id': {u'city': u'TANANA', u'state': u'AK'}, u'pop': 345}
{u'_id': {u'city': u'SHAKTOOLIK', u'state': u'AK'}, u'pop': 183}
{u'_id': {u'city': u'SELAWIK', u'state': u'AK'}, u'pop': 0}
{u'_id': {u'city': u'SAVOONGA', u'state': u'AK'}, u'pop': 519

{u'_id': {u'city': u'GLENWOOD', u'state': u'WA'}, u'pop': 549}
{u'_id': {u'city': u'CENTERVILLE', u'state': u'WA'}, u'pop': 681}
{u'_id': {u'city': u'CATHLAMET', u'state': u'WA'}, u'pop': 2114}
{u'_id': {u'city': u'CARSON', u'state': u'WA'}, u'pop': 2009}
{u'_id': {u'city': u'CAMAS', u'state': u'WA'}, u'pop': 12058}
{u'_id': {u'city': u'ARIEL', u'state': u'WA'}, u'pop': 735}
{u'_id': {u'city': u'WINLOCK', u'state': u'WA'}, u'pop': 5206}
{u'_id': {u'city': u'WESTPORT', u'state': u'WA'}, u'pop': 2463}
{u'_id': {u'city': u'VADER', u'state': u'WA'}, u'pop': 523}
{u'_id': {u'city': u'TOKELAND', u'state': u'WA'}, u'pop': 891}
{u'_id': {u'city': u'TAHUYA', u'state': u'WA'}, u'pop': 3794}
{u'_id': {u'city': u'TAHOLAH', u'state': u'WA'}, u'pop': 851}
{u'_id': {u'city': u'SOUTH BEND', u'state': u'WA'}, u'pop': 2575}
{u'_id': {u'city': u'SHELTON', u'state': u'WA'}, u'pop': 19074}
{u'_id': {u'city': u'PE ELL', u'state': u'WA'}, u'pop': 921}
{u'_id': {u'city': u'ONALASKA', u'state': u'WA'}, u'pop':

{u'_id': {u'city': u'COQUILLE', u'state': u'OR'}, u'pop': 8374}
{u'_id': {u'city': u'CHARLESTON', u'state': u'OR'}, u'pop': 24343}
{u'_id': {u'city': u'CHESHIRE', u'state': u'OR'}, u'pop': 1050}
{u'_id': {u'city': u'CANYONVILLE', u'state': u'OR'}, u'pop': 1670}
{u'_id': {u'city': u'AZALEA', u'state': u'OR'}, u'pop': 1111}
{u'_id': {u'city': u'WILLAMINA', u'state': u'OR'}, u'pop': 2697}
{u'_id': {u'city': u'TURNER', u'state': u'OR'}, u'pop': 1243}
{u'_id': {u'city': u'TOLEDO', u'state': u'OR'}, u'pop': 5609}
{u'_id': {u'city': u'TIDEWATER', u'state': u'OR'}, u'pop': 523}
{u'_id': {u'city': u'SUBLIMITY', u'state': u'OR'}, u'pop': 2881}
{u'_id': {u'city': u'SCOTTS MILLS', u'state': u'OR'}, u'pop': 1326}
{u'_id': {u'city': u'PHILOMATH', u'state': u'OR'}, u'pop': 6879}
{u'_id': {u'city': u'LINCOLN CITY', u'state': u'OR'}, u'pop': 6786}
{u'_id': {u'city': u'MOUNT ANGEL', u'state': u'OR'}, u'pop': 4747}
{u'_id': {u'city': u'MONMOUTH', u'state': u'OR'}, u'pop': 8071}
{u'_id': {u'city': u'LOGSD

{u'_id': {u'city': u'FELTON', u'state': u'CA'}, u'pop': 8194}
{u'_id': {u'city': u'MONTE VISTA', u'state': u'CA'}, u'pop': 47598}
{u'_id': {u'city': u'CASTROVILLE', u'state': u'CA'}, u'pop': 7168}
{u'_id': {u'city': u'AROMAS', u'state': u'CA'}, u'pop': 2713}
{u'_id': {u'city': u'OLALLA', u'state': u'WA'}, u'pop': 3173}
{u'_id': {u'city': u'APTOS', u'state': u'CA'}, u'pop': 23964}
{u'_id': {u'city': u'BONNEVILLE', u'state': u'OR'}, u'pop': 951}
{u'_id': {u'city': u'VALLEY FORD', u'state': u'CA'}, u'pop': 114}
{u'_id': {u'city': u'POINT REYES STAT', u'state': u'CA'}, u'pop': 951}
{u'_id': {u'city': u'NICASIO', u'state': u'CA'}, u'pop': 665}
{u'_id': {u'city': u'NOVATO', u'state': u'CA'}, u'pop': 54513}
{u'_id': {u'city': u'MARSHALL', u'state': u'CA'}, u'pop': 55}
{u'_id': {u'city': u'LARKSPUR', u'state': u'CA'}, u'pop': 5884}
{u'_id': {u'city': u'FOREST KNOLLS', u'state': u'CA'}, u'pop': 732}
{u'_id': {u'city': u'FAIRFAX', u'state': u'CA'}, u'pop': 8051}
{u'_id': {u'city': u'CORTE MADERA

{u'_id': {u'city': u'INDIAN WELLS', u'state': u'CA'}, u'pop': 2599}
{u'_id': {u'city': u'VALLEY CENTER', u'state': u'CA'}, u'pop': 13196}
{u'_id': {u'city': u'SANTEE', u'state': u'CA'}, u'pop': 52816}
{u'_id': {u'city': u'SAN MARCOS', u'state': u'CA'}, u'pop': 45382}
{u'_id': {u'city': u'RANCHITA', u'state': u'CA'}, u'pop': 389}
{u'_id': {u'city': u'RAMONA', u'state': u'CA'}, u'pop': 27744}
{u'_id': {u'city': u'PAUMA VALLEY', u'state': u'CA'}, u'pop': 1929}
{u'_id': {u'city': u'MARINE CORP BASE', u'state': u'CA'}, u'pop': 13643}
{u'_id': {u'city': u'OCEANSIDE', u'state': u'CA'}, u'pop': 135099}
{u'_id': {u'city': u'WINDSOR', u'state': u'CA'}, u'pop': 13717}
{u'_id': {u'city': u'LAKESIDE', u'state': u'CA'}, u'pop': 41054}
{u'_id': {u'city': u'LA JOLLA', u'state': u'CA'}, u'pop': 40399}
{u'_id': {u'city': u'JULIAN', u'state': u'CA'}, u'pop': 2552}
{u'_id': {u'city': u'FALLBROOK', u'state': u'CA'}, u'pop': 35232}
{u'_id': {u'city': u'ESCONDIDO', u'state': u'CA'}, u'pop': 134000}
{u'_id': 

...

{u'_id': {u'city': u'BUFFALO', u'state': u'MT'}, u'pop': 105}
{u'_id': {u'city': u'WALLACE', u'state': u'MI'}, u'pop': 2103}
{u'_id': {u'city': u'WASHTA', u'state': u'IA'}, u'pop': 519}
{u'_id': {u'city': u'ASHLAND', u'state': u'MO'}, u'pop': 2989}
{u'_id': {u'city': u'LORRAINE', u'state': u'NY'}, u'pop': 462}
{u'_id': {u'city': u'SIOUX CITY', u'state': u'IA'}, u'pop': 82049}
{u'_id': {u'city': u'SHELDON', u'state': u'IA'}, u'pop': 5585}
{u'_id': {u'city': u'ALVORD', u'state': u'IA'}, u'pop': 494}
{u'_id': {u'city': u'ASHTON', u'state': u'IA'}, u'pop': 1090}
{u'_id': {u'city': u'SOUDER', u'state': u'MO'}, u'pop': 215}
{u'_id': {u'city': u'PENNSBURG', u'state': u'PA'}, u'pop': 5126}
{u'_id': {u'city': u'COTTAGEVILLE', u'state': u'SC'}, u'pop': 4105}
{u'_id': {u'city': u'DOON', u'state': u'IA'}, u'pop': 957}
{u'_id': {u'city': u'HOSPERS', u'state': u'IA'}, u'pop': 1491}
{u'_id': {u'city': u'INWOOD', u'state': u'IA'}, u'pop': 1434}
{u'_id': {u'city': u'BROWNSVILLE', u'state': u'TX'}, u'po

{u'_id': {u'city': u'ROYALTON', u'state': u'MN'}, u'pop': 2571}
{u'_id': {u'city': u'GREENBACK', u'state': u'TN'}, u'pop': 3272}
{u'_id': {u'city': u'GREELEY', u'state': u'IA'}, u'pop': 679}
{u'_id': {u'city': u'MILES', u'state': u'IA'}, u'pop': 1083}
{u'_id': {u'city': u'PALMYRA', u'state': u'NY'}, u'pop': 9486}
{u'_id': {u'city': u'LANCASTER', u'state': u'NH'}, u'pop': 3825}
{u'_id': {u'city': u'PRESTON', u'state': u'IA'}, u'pop': 1376}
{u'_id': {u'city': u'SABULA', u'state': u'IA'}, u'pop': 1174}
{u'_id': {u'city': u'ZWINGLE', u'state': u'IA'}, u'pop': 1320}
{u'_id': {u'city': u'COLONY', u'state': u'OK'}, u'pop': 312}
{u'_id': {u'city': u'ROCKAWAY BEACH', u'state': u'MO'}, u'pop': 1354}
{u'_id': {u'city': u'BURR OAK', u'state': u'IA'}, u'pop': 484}
{u'_id': {u'city': u'CASTALIA', u'state': u'IA'}, u'pop': 1103}
{u'_id': {u'city': u'CHAPIN', u'state': u'SC'}, u'pop': 8744}
{u'_id': {u'city': u'CLERMONT', u'state': u'IA'}, u'pop': 817}
{u'_id': {u'city': u'FAYETTE', u'state': u'IA'}, 

{u'_id': {u'city': u'QUANTICO', u'state': u'VA'}, u'pop': 9039}
{u'_id': {u'city': u'BONO', u'state': u'OH'}, u'pop': 3895}
{u'_id': {u'city': u'BAILEYS HARBOR', u'state': u'WI'}, u'pop': 1167}
{u'_id': {u'city': u'BRUSSELS', u'state': u'WI'}, u'pop': 1785}
{u'_id': {u'city': u'CASCO', u'state': u'WI'}, u'pop': 2066}
{u'_id': {u'city': u'FRANCIS CREEK', u'state': u'WI'}, u'pop': 562}
{u'_id': {u'city': u'GLENS FORK', u'state': u'KY'}, u'pop': 788}
{u'_id': {u'city': u'STURGEON BAY', u'state': u'WI'}, u'pop': 16149}
{u'_id': {u'city': u'OAK FOREST', u'state': u'IL'}, u'pop': 26772}
{u'_id': {u'city': u'VALDERS', u'state': u'WI'}, u'pop': 2056}
{u'_id': {u'city': u'GREEN BAY', u'state': u'WI'}, u'pop': 70006}
{u'_id': {u'city': u'HOWARD', u'state': u'WI'}, u'pop': 27046}
{u'_id': {u'city': u'ASHWAUBENON', u'state': u'WI'}, u'pop': 31339}
{u'_id': {u'city': u'WEST SHOKAN', u'state': u'NY'}, u'pop': 809}
{u'_id': {u'city': u'ABBOTSFORD', u'state': u'WI'}, u'pop': 2480}
{u'_id': {u'city': u

{u'_id': {u'city': u'ROCKLAND', u'state': u'WI'}, u'pop': 1486}
{u'_id': {u'city': u'STEUBEN', u'state': u'WI'}, u'pop': 63}
{u'_id': {u'city': u'EMRICK', u'state': u'ND'}, u'pop': 115}
{u'_id': {u'city': u'STODDARD', u'state': u'WI'}, u'pop': 2116}
{u'_id': {u'city': u'WYEVILLE', u'state': u'WI'}, u'pop': 14294}
{u'_id': {u'city': u'SCOTTVILLE', u'state': u'MI'}, u'pop': 4229}
{u'_id': {u'city': u'VIROQUA', u'state': u'WI'}, u'pop': 7071}
{u'_id': {u'city': u'BETHEL', u'state': u'VT'}, u'pop': 2725}
{u'_id': {u'city': u'EAU CLAIRE', u'state': u'WI'}, u'pop': 69993}
{u'_id': {u'city': u'MARKLEYSBURG', u'state': u'PA'}, u'pop': 2052}
{u'_id': {u'city': u'BAY CITY', u'state': u'WI'}, u'pop': 1540}
{u'_id': {u'city': u'CRESCENT', u'state': u'OR'}, u'pop': 1848}
{u'_id': {u'city': u'PULASKI', u'state': u'IL'}, u'pop': 663}
{u'_id': {u'city': u'BLOOMER', u'state': u'WI'}, u'pop': 6410}
{u'_id': {u'city': u'BOYCEVILLE', u'state': u'WI'}, u'pop': 2520}
{u'_id': {u'city': u'TINNIE', u'state': 

{u'_id': {u'city': u'NORTHROP', u'state': u'MN'}, u'pop': 724}
{u'_id': {u'city': u'AVON', u'state': u'NY'}, u'pop': 6187}
{u'_id': {u'city': u'PEMBERTON', u'state': u'MN'}, u'pop': 880}
{u'_id': {u'city': u'SAINT CLAIR', u'state': u'MN'}, u'pop': 1083}
{u'_id': {u'city': u'TRUMAN', u'state': u'MN'}, u'pop': 2319}
{u'_id': {u'city': u'WALDORF', u'state': u'MN'}, u'pop': 962}
{u'_id': {u'city': u'STRAUGHN', u'state': u'IN'}, u'pop': 1300}
{u'_id': {u'city': u'WALTERS', u'state': u'MN'}, u'pop': 400}
{u'_id': {u'city': u'WASECA', u'state': u'MN'}, u'pop': 11534}
{u'_id': {u'city': u'FOREST JUNCTION', u'state': u'WI'}, u'pop': 395}
{u'_id': {u'city': u'WATERVILLE', u'state': u'MN'}, u'pop': 2390}
{u'_id': {u'city': u'WELLS', u'state': u'MN'}, u'pop': 3585}
{u'_id': {u'city': u'BROOKFIELD', u'state': u'VT'}, u'pop': 453}
{u'_id': {u'city': u'WINNEBAGO', u'state': u'MN'}, u'pop': 1998}
{u'_id': {u'city': u'JAMAICA', u'state': u'VA'}, u'pop': 383}
{u'_id': {u'city': u'WILDER', u'state': u'MN

A second \$group stage groups the documents in the pipeline by the _id.state field (i.e. the state field inside the _id document), uses the \$avg expression to calculate the average city population (avgCityPop) for each state, and outputs a document for each state.

In [33]:
group_stage2 = {
    "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" }}
}

sort_stage = {
    "$sort": {'_id': pymongo.DESCENDING}
}


pipeline = [group_stage1, group_stage2, sort_stage]

for doc in collection.aggregate(pipeline):
    pprint(doc)


{u'_id': u'WY', u'avgCityPop': 3384.5373134328356}
{u'_id': u'WV', u'avgCityPop': 2771.4775888717154}
{u'_id': u'WI', u'avgCityPop': 7323.00748502994}
{u'_id': u'WA', u'avgCityPop': 12258.670025188916}
{u'_id': u'VT', u'avgCityPop': 2315.8765432098767}
{u'_id': u'VA', u'avgCityPop': 8526.177931034483}
{u'_id': u'UT', u'avgCityPop': 9518.508287292818}
{u'_id': u'TX', u'avgCityPop': 13775.02108678021}
{u'_id': u'TN', u'avgCityPop': 9656.350495049504}
{u'_id': u'SD', u'avgCityPop': 1839.6746031746031}
{u'_id': u'SC', u'avgCityPop': 11139.626198083068}
{u'_id': u'RI', u'avgCityPop': 19292.653846153848}
{u'_id': u'PA', u'avgCityPop': 8679.067202337472}
{u'_id': u'OR', u'avgCityPop': 8262.561046511628}
{u'_id': u'OK', u'avgCityPop': 6155.743639921722}
{u'_id': u'OH', u'avgCityPop': 12700.839578454332}
{u'_id': u'NY', u'avgCityPop': 13131.680291970803}
{u'_id': u'NV', u'avgCityPop': 18209.590909090908}
{u'_id': u'NM', u'avgCityPop': 5872.360465116279}
{u'_id': u'NJ', u'avgCityPop': 15775.8938

### Return Largest and Smallest Cities by State


In [34]:
group_stage1 = {
    "$group": { "_id": {"state": "$state", "city": "$city"}, "pop": { "$sum": "$pop" }}
}

sort_stage = {
    "$sort": {"pop": 1}
}

group_stage2 = {
    "$group": {"_id": "$_id.state", 
                   "biggestCity": {"$last": "$_id.city"},
                   "biggestPop": {"$last": "$pop"},
                   "smallestCity": {"$first": "$_id.city"},
                   "smallestPop": {"$first": "$pop"},
              
              }
}

project_stage = {
    "$project": {
        "_id": 0,
        "state:": "$_id",
        "biggestCityData": {"name": "$biggestCity", "pop": "$biggestPop"},
        "smallestCityData": {"name": "$smallestCity", "pop": "$smallestPop"}
    }
}


pipeline = [group_stage1, sort_stage, group_stage2, project_stage]

for doc in collection.aggregate(pipeline):
    pprint(doc)



{u'biggestCityData': {u'name': u'INDIANAPOLIS', u'pop': 348868},
 u'smallestCityData': {u'name': u'WESTPOINT', u'pop': 145},
 u'state:': u'IN'}
{u'biggestCityData': {u'name': u'NEWARK', u'pop': 111674},
 u'smallestCityData': {u'name': u'BETHEL', u'pop': 108},
 u'state:': u'DE'}
{u'biggestCityData': {u'name': u'JACKSON', u'pop': 204788},
 u'smallestCityData': {u'name': u'CHUNKY', u'pop': 79},
 u'state:': u'MS'}
{u'biggestCityData': {u'name': u'CRANSTON', u'pop': 176404},
 u'smallestCityData': {u'name': u'CLAYVILLE', u'pop': 45},
 u'state:': u'RI'}
{u'biggestCityData': {u'name': u'ATLANTA', u'pop': 609591},
 u'smallestCityData': {u'name': u'FORT STEWART', u'pop': 0},
 u'state:': u'GA'}
{u'biggestCityData': {u'name': u'WASHINGTON', u'pop': 606879},
 u'smallestCityData': {u'name': u'PENTAGON', u'pop': 21},
 u'state:': u'DC'}
{u'biggestCityData': {u'name': u'HUNTINGTON', u'pop': 75343},
 u'smallestCityData': {u'name': u'MOUNT CARBON', u'pop': 0},
 u'state:': u'WV'}
{u'biggestCityData': {u'n

### Another link to check out:

[link](https://docs.mongodb.com/manual/tutorial/aggregation-with-user-preference-data/)

This is a simple query, which shows some of the basic stages of the aggregation pipeline.  It can be improved as follows:

* We can change the name of the `_id` in the output back to `cuisine` using the `$project` stage
* We can change the order of the output to be sorted in alphabetical order as well
* We can limit the results to include results only with a count of 20 or more

Implement those stages in the cell below

In [35]:
collection = db.restaurants

In [36]:
collection.find_one()

{u'_id': ObjectId('5c0bd192d7994444426c6afe'),
 u'address': {u'building': u'1007',
  u'coord': [-73.856077, 40.848447],
  u'street': u'Morris Park Ave',
  u'zipcode': u'10462'},
 u'borough': u'Bronx',
 u'cuisine': u'Bakery',
 u'grades': [{u'date': datetime.datetime(2014, 3, 3, 0, 0),
   u'grade': u'A',
   u'score': 2},
  {u'date': datetime.datetime(2013, 9, 11, 0, 0), u'grade': u'A', u'score': 6},
  {u'date': datetime.datetime(2013, 1, 24, 0, 0),
   u'grade': u'A',
   u'score': 10},
  {u'date': datetime.datetime(2011, 11, 23, 0, 0),
   u'grade': u'A',
   u'score': 9},
  {u'date': datetime.datetime(2011, 3, 10, 0, 0),
   u'grade': u'B',
   u'score': 14}],
 u'name': u'Morris Park Bake Shop',
 u'restaurant_id': u'30075445'}

In [37]:
# YOUR CODE HERE FOR sort

group = {
    '$group': {'_id': '$cuisine', 'count': {'$sum': 1}}
    
}

sort = {
    '$sort': {'count': pymongo.DESCENDING}
}

match_final = {
    "$match": {"count": {"$gt": 100}}
}



pipeline.append(sort)
cursor = collection.aggregate(pipeline)
for c in cursor:
    pprint(c)


{u'biggestCityData': {u'name': None, u'pop': 0},
 u'smallestCityData': {u'name': None, u'pop': 0},
 u'state:': None}


In [38]:
# YOUR CODE HERE FOR `count` > 20

cursor = collection.aggregate(pipeline)
for c in cursor:
    pprint(c)


{u'biggestCityData': {u'name': None, u'pop': 0},
 u'smallestCityData': {u'name': None, u'pop': 0},
 u'state:': None}


## Challenge

How would you work out the percentage of each type of cuisine out of all selected restaurants?

In [39]:

group_stage1 = {
    "$group": {"_id": None, "total": {"$sum": 1}}
}

total_cuisines = []

pipeline = [group_stage1]

for doc in collection.aggregate(pipeline):
    total_cuisines.append((doc['total']))

In [40]:
total_cuisines[0]

25359

In [41]:
group_stage2 = {
    "$group": {"_id": "$cuisine", "total": {"$sum": 1}}
}

project_stage1 = {
    "$project": {
    "name": "$_id",
    "_id": 0,
    "total": {"$divide": [{"$multiply": ["$total", 100]}, total_cuisines[0]]}
    }
}


pipeline = [group_stage2, project_stage1]
for doc in collection.aggregate(pipeline):
    pprint(doc)

{u'name': u'Chilean', u'total': 0.003943373161402263}
{u'name': u'Californian', u'total': 0.003943373161402263}
{u'name': u'Creole/Cajun', u'total': 0.003943373161402263}
{u'name': u'Nuts/Confectionary', u'total': 0.023660238968413582}
{u'name': u'Southwestern', u'total': 0.03549035845262037}
{u'name': u'Hotdogs/Pretzels', u'total': 0.06309397058243621}
{u'name': u'Australian', u'total': 0.06309397058243621}
{u'name': u'Soups', u'total': 0.015773492645609052}
{u'name': u'Polynesian', u'total': 0.003943373161402263}
{u'name': u'Cajun', u'total': 0.027603612129815845}
{u'name': u'Bangladeshi', u'total': 0.14196143381048149}
{u'name': u'Peruvian', u'total': 0.2681493749753539}
{u'name': u'Vietnamese/Cambodian/Malaysia', u'total': 0.2602626286525494}
{u'name': u'Indonesian', u'total': 0.031546985291218105}
{u'name': u'Portuguese', u'total': 0.031546985291218105}
{u'name': u'Salads', u'total': 0.17745179226310187}
{u'name': u'Sandwiches', u'total': 1.8100082810836389}
{u'name': u'Egyptian',

# Where next?

* Dealing with array data
* Update, delete, and drop
* Setting up authentication
* Sharding databases